In [ ]:
pip install gtts

In [ ]:
pip install pygame

In [ ]:
import transformers
import os
from gtts import gTTS
import pygame
import tensorflow as tf

Execution of the chatbot

In [ ]:
class ChatBot:
    def __init__(self, name):
        print("----- Starting up", name, "-----")
        self.name = name
        pygame.init()
        # pygame.mixer.init()  # Uncomment if needed

    def speak(self, text):
        tts = gTTS(text=text, lang='en')
        tts.save("output.mp3")
        pygame.mixer.music.load("output.mp3")
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            continue

    def wake_up(self, text):
        return True if self.name.lower() in text.lower() else False

if __name__ == "__main__":
    ai = ChatBot(name="GP")
    token = transformers.AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    nlp = transformers.pipeline("conversational", model="microsoft/DialoGPT-medium", tokenizer=token, device='cpu')
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    ex = True
    while ex:
        ai.text = input("You: ")  # Get user input inside the loop
        if ai.wake_up(ai.text):
            res = "Hello I am " + ai.name + ", what can I do for you?"
            ai.speak(res)
        elif any(i in ai.text for i in ["exit", "close", "bye"]):
            res = "Tata"  # Use a default exit message for simplicity
            ai.speak(res)
            ex = False
        else:
            try:
                chat = nlp(transformers.Conversation(ai.text), pad_token_id=50256)
                response = chat.generated_responses[0]
                print("Bot:", response)  # Print the bot's response directly
                ai.speak(response)
            except KeyError as e:
                print("Error:", e)
            except Exception as e:
                print("Unexpected error:", e)
    print("----- Closing down chatbot -----")

